In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from tqdm.notebook import tqdm

In [3]:
def mse(img): # Error between input image and white image
  return np.sum((255-img)**2)/(np.shape(img)[0]*np.shape(img)[1])
  
def is_pushed(img): # image is pushed image if error is lager than threshold(200) (check image is almost white)
                    # when not pushed, image is almost white
  return 1 if mse(img) > 200 else 0

In [11]:
# 저장 path 설정
drivepath = "/content/gdrive/MyDrive/"
vidname = "kart_test.mp4"                            # video 파일 이름
capdir = f"cap2_{vidname}/"                # 이미지 저장 directory 이름
csvname = f"{vidname}.csv"                # csv 파일 이름
cappath = os.path.join(drivepath,capdir)  # 이미지 저장 경로
csvpath = os.path.join(drivepath, "csv/")  # csv 파일 경로
vidpath = os.path.join(drivepath, vidname)

# directory 없을 시 생성
if not os.path.exists(cappath):
  os.makedirs(cappath)

if not os.path.exists(csvpath):
  os.makedirs(csvpath)

In [12]:
# csv, 비디오 파일 open
cap = cv2.VideoCapture(vidpath)
fcsv = open(os.path.join(csvpath+csvname),'w',encoding='utf-8')
wrcsv = csv.writer(fcsv)

영상 추출 관련 기본 데이터

In [13]:
fps = round(cap.get(cv2.CAP_PROP_FPS))
start_sec = 105          # Start time
end_sec = 110   # End time
interval = 0.1    # Time interval
# 단위는 초

print(f"fps : {fps}")

fps : 60


In [14]:
# top left position of key
# up, left, right, ctrl, shift, down 
key_pos = [(1265,914), (1188,992), (1343,992), (1092,992), (1092,914), (1265,992)]

# 키 인식 이미지 크기
key_size = 20

prevpath = ""

In [17]:
for t in tqdm(np.arange(start_sec, end_sec, interval)):
  cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
  ret,frame = cap.read()

  if not ret:
    print("영상 시간 초과 또는 영상 관련 오류")

  # 키 입력 추출
  key_array = [is_pushed(frame[index[1]:index[1]+key_size,index[0]:index[0]+key_size]) for index in key_pos]

  # 추출된 키 입력을 string type 변경
  keystring = ''.join(str(key) for key in key_array)
  #print(f"key string at {t} is {keystring}")

  # 게임 화면만 저장
  cropped_img = frame[:1080,:1440]

  # 이미지 파일에서 키 뷰어부분 마스킹
  frame[881:1080,1056:1440] = 0

  # 이미지 저장 경로
  imgpath = os.path.join(capdir,f"t{t}.png")

  # 이미지 저장
  cv2.imwrite(os.path.join(drivepath,imgpath), cropped_img)

  # csv 파일 저장 (이미지 저장 경로, 이전 입력)
  wrcsv.writerow([prevpath,keystring])

  t += interval
  prevpath = imgpath

In [18]:
cap.release()
fcsv.close()